In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from skimage import io
from scipy.stats import describe
from skimage.exposure import rescale_intensity, histogram, adjust_gamma
from flowdec import data as fd_data
import pims
import pims
from skimage import io
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
import matplotlib.pyplot as plt

/home/jmamede/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jmamede/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jmamede/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jmamede/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

In [2]:
channels = ['a647', 'mRuby','GFP']


In [3]:


# frames =  pims.ND2_Reader('/home/jmamede/Area51/igfp3_caruby3_rutin_bvd_f2_2days_xy02.nd2')
frames =  ND2Reader('/home/jmamede/Area51/antibody4.nd2')
#frames.iter_axes = 't'  # 't' is the default already
frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default

frames.sizes
#['z']
#frames.bundle_axes = 'czyx'  # when 'z' is available, this will be default
#frames.iter_axes = 'c' 
#frames.default_coords['c'] = 1  # 0 is the default setting
#for frame in frames[:3]:
    # do something with 3D frames in channel 1
    

{'x': 2048, 'y': 2044, 'c': 5, 't': 1, 'z': 11, 'v': 8}

In [4]:
meta = frames.metadata
print(meta)
NS_BINARY_FILE = "http://www.openmicroscopy.org/Schemas/BinaryFile/2013-06"
NS_ORIGINAL_METADATA = "openmicroscopy.org/OriginalMetadata"
NS_DEFAULT = "http://www.openmicroscopy.org/Schemas/{ns_key}/2013-06"
NS_RE = r"http://www.openmicroscopy.org/Schemas/(?P<ns_key>.*)/[0-9/-]"

default_xml = """<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
     xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
     xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0" Name="default.png">
        <AcquisitionDate>%(DEFAULT_NOW)s</AcquisitionDate>
        <Pixels BigEndian="false"
                DimensionOrder="XYCZT"
                ID="Pixels:0"
                Interleaved="false"
                SizeC="2"
                SizeT="1"
                SizeX="2044"
                SizeY="2044"
                SizeZ="11"
                Type="uint8">
            <Channel ID="Channel:0:0" SamplesPerPixel="1">
                <LightPath/>
            </Channel>
        </Pixels>
    </Image>
  <StructuredAnnotations xmlns="{ns_sa_default}s"/>
</OME>""".format(ns_ome_default=NS_DEFAULT.format(ns_key='ome'), ns_sa_default=NS_DEFAULT.format(ns_key='sa'))

{'height': 2044, 'width': 2048, 'date': datetime.datetime(2020, 6, 11, 15, 14, 6), 'fields_of_view': range(0, 8), 'frames': [0], 'z_levels': range(0, 11), 'z_coordinates': [4223.92, 4224.22, 4224.52, 4224.82, 4225.16, 4225.42, 4225.72, 4226.02, 4226.3, 4226.62, 4226.92, 4223.18, 4223.4800000000005, 4223.78, 4224.1, 4224.38, 4224.68, 4224.96, 4225.28, 4225.58, 4225.88, 4226.18, 4222.5, 4222.8, 4223.1, 4223.4, 4223.7, 4224.0, 4224.3, 4224.6, 4224.92, 4225.2, 4225.5, 4222.2, 4222.5, 4222.76, 4223.1, 4223.38, 4223.7, 4224.0, 4224.24, 4224.6, 4224.9, 4225.2, 4222.4, 4222.7, 4223.04, 4223.28, 4223.6, 4223.88, 4224.2, 4224.5, 4224.8, 4225.12, 4225.38, 4223.54, 4223.84, 4224.14, 4224.42, 4224.74, 4225.04, 4225.34, 4225.64, 4225.9400000000005, 4226.22, 4226.52, 4223.76, 4224.08, 4224.36, 4224.66, 4224.96, 4225.24, 4225.58, 4225.88, 4226.16, 4226.4800000000005, 4226.76, 4224.24, 4224.52, 4224.84, 4225.14, 4225.42, 4225.72, 4226.04, 4226.34, 4226.64, 4226.96, 4227.22], 'total_images_per_channel':

In [5]:
psf =[]
#psf = fd_psf.GibsonLanni.load('/tmp/psf.json')

# args = [na=1.4, wavelength= 0.594 , size_z=  11, size_x=  2048, size_y=  2044,
#         m :60,ns =  1.333, ni =  1.5156, tg =  170,         
#         res_lateral =  0.108333333333333, res_axial =  0.3, pZ = 1]

# args = [{"na": 1.4, "wavelength": '0.594' ,
#          "size_z": 11, "size_x": 2048, "size_y": 2044,
#          "m" :60,"ns" : 1.333, "ni" : 1.5156, "tg" : 170,
#          "res_lateral" : 0.108333333333333, "res_axial" : 0.3,
#          "pZ" :1}]

#na = meta['NA']
na = 1.4
#ra = meta['refractive_index1']
ra = 1.5156
z = frames.sizes['z']
x = 512
y = 512
res_z = 0.5
res_xy =  meta['pixel_microns']

psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.670 , size_z=  z, size_x=  x , size_y=  y
    ,m =60, ns =  1.333
    , tg =  170, tg0 = 170
    , ni0 =  ra, ni =  ra
    ,res_lateral =  res_xy , res_axial = res_z, 
    #pz = 1
)
psf647 = psf.generate()    
    
psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.620 , size_z=  z, size_x=  x , size_y=  y
    ,m =60, ns =  1.333
    , tg =  170, tg0 = 170
    , ni0 =  ra, ni =  ra
    ,res_lateral =  res_xy , res_axial = res_z, 
    #pz = 1
)
psfruby = psf.generate()

psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.535 , size_z=  z, size_x=  x , size_y=  y
    ,m =60, ns =  1.333
    , tg =  170, tg0 = 170
    , ni0 =  ra, ni =  ra
    ,res_lateral =  res_xy , res_axial = res_z, 
    #pz = 1
)
psfgfp = psf.generate()

psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.420 , size_z=  z, size_x=  x , size_y=  y
    ,m =60, ns =  1.333
    , tg =  170, tg0 = 170
    , ni0 =  ra, ni =  ra
    ,res_lateral =  res_xy , res_axial = res_z, 
    #pz = 1
)
psfdapi = psf.generate()  

# from pims import Frame
# Frame(psfdapi)
print(psf647.shape)
print(frames[0][:,512:1536,512:1536].shape)

(11, 1022, 1022)
(11, 1024, 1024)


In [6]:
#ms = []
def observer(img, i, *args):
    #mgs.append(img.max(axis=0))
    if i % 5 == 0:
        print('Observing iteration = {} (dtype = {}, max = {:.3f})'.format(i, img.dtype, img.max()))   
#config = tf.ConfigProto(device_count={'GPU': 1})
#algo = fd_restoration.RichardsonLucyDeconvolver(n_dims=acq.data.ndim, pad_min=[1, 1, 1], session_config=config).initialize()
algo = fd_restoration.RichardsonLucyDeconvolver(n_dims=psfgfp.ndim
                                                #, pad_mode='none'
                                                #, pad_mode='none'
                                                #, pad_min=[1,1,1]
                                                #,pad_min=np.ones(psfdapi.ndim)
                                                ,observer_fn=observer
                                                #,real_domain_fft=True
                                                #,device='/cpu:0'
                                                
                                               ).initialize()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
# import datetime

import tifffile as tf
import tensorflow as tflow
# import json

chunk_size=(frames.sizes['z'],512,512)

def deconv(chunk):
    # note that algo and cropped_kernel are from global scope ... ugly
    print("chunk shape", chunk.shape)
    tmp = algo.initialize().run(fd_data.Acquisition(data=chunk, kernel=psf)
#                                 , session_config=tflow.compat.v1.ConfigProto(
#                                 device_count={'GPU': 1}        
#                                 , GPUOptions={'allow_growth' : 4}
#                                         )
                                ,niter=15
                               )
    return tmp.data

for visit in range(frames.sizes['v']):
    #%timeit
    with tf.TiffWriter('/home/jmamede/Area51/'+str(visit)+'.tif'
               #, bigtiff=True
               #, imagej=True
                      ) as tif:
        print('Visit:',visit)
        frames.default_coords['v'] = visit

        for time in range(frames.sizes['t']):
            print('Time:',time)
            frames.default_coords['t'] = time

#             for ch in channels:
            for i in range(frames.sizes['c']):
                print('Channel:',meta['channels'][i])
                ch = meta['channels'][i]
                if ch == 'DAPI':
                    frames.default_coords['c'] = i
                    #split into 4 arrays of 1022,1022
                    arr = da.from_array(frames[0][:,0:2044,0:2044], chunks=chunk_size)
                    
                    psf = psfdapi
#make mem efficient by creating and empty array with the right shape with all channels and then add it as it goes.
                    res0 = arr.map_overlap(
                        deconv,depth=(0,6,6),
                        boundary='reflect',
                        dtype='float32').compute(num_workers=1)
#                     res0 = arr.map_blocks(
#                         deconv,dtype='float32').compute(num_workers=1)

                elif ch == 'mRuby': 
                    frames.default_coords['c'] = i
                    arr = da.from_array(frames[0][:,0:2044,0:2044], chunks=chunk_size)
                    psf = psfruby

                    res1 =arr.map_overlap(
                        deconv,depth=(0,6,6),
                        boundary='reflect',
                        dtype='float32').compute(num_workers=1)
#                     res1 = arr.map_blocks(
#                         deconv,dtype='float32').compute(num_workers=1)
#                 elif ch == 'GFP':
#                     frames.default_coords['c'] = i
# #                     psf = psfgfp
#                     res2 =algo.run(
#                     fd_data.Acquisition(
#                     data=frames[0],
#                     kernel=psfgfp)
#                     , niter=15)


            img5d = np.stack((res0, res1
#                               ,res2.data
                             ),axis=0).astype(np.int16)
        #     img5d = np.expand_dims(img5d, axis=0).astype(np.int16)
        # swap color and Z
            img5d = np.swapaxes(img5d,0,1)

            tif.save(img5d
                    ,shape=img5d.shape

            #,resolution= (.1083,0.1083,3)
             , description = default_xml
            , photometric='minisblack'
            #, datetime= True
            , metadata={'axes': 'TZCYX', 'DimensionOrder' : 'TZCYX'}
                )
        tif.close()
#             res.append(algo.run(
#                 fd_data.Acquisition(
#                 data=frames[0],
#                 kernel=psf)
#                 , niter=20)
#                       )
            #reassumblethetiff.
            


Visit: 0
Time: 0
Channel: DAPI
chunk shape (0, 0, 0)
chunk shape (11, 1034, 1034)


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[16,2048,2048] and type complex64 on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node while/FFT3D (defined at /home/jmamede/anaconda3/lib/python3.7/site-packages/flowdec/restoration.py:293) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[result/_165]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[16,2048,2048] and type complex64 on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node while/FFT3D (defined at /home/jmamede/anaconda3/lib/python3.7/site-packages/flowdec/restoration.py:293) ]]
Hint: If you want to see a list of allocated tensors when O

In [20]:
algo.
# tif.close()
from scipy.stats import describe
describe(frames[0].ravel())


DescribeResult(nobs=46047232, minmax=(103.0, 10929.0), mean=350.4336648726247, variance=31078.272057750954, skewness=5.981472775982217, kurtosis=84.91737559687797)

In [ ]:
plt.imshow(frames[0].max(axis=0),vmax=5000)

In [ ]:
res_stack = np.stack((res[1].data,np.zeros(res[1].data.shape), res[0].data))

In [ ]:
print(res_stack.shape)

In [ ]:
plt.imshow(res[1].data.max(axis=0),vmax=5000)

In [ ]:
plt.imshow(res[0].data[10],vmax=5000)

In [ ]:
from pims import Frame

zproj = res.data.max(axis=0)
frame = Frame(zproj
    )


#frame = Frame(res.data)
%matplotlib qt5

In [ ]:
plt.imshow(frame)

In [ ]:
plt.imshow(psf[0])

In [ ]:
print(meta)

In [ ]:
ctrl = pims.ND2_Reader('/home/jmamede/Area51/antibody4 - Deconvolved, Type Automatic.nd2')
ctrl.bundle_axes = 'zyx'
ctrl.default_coords['m'] = 0

print(ctrl[0].shape)
#frames.bundle_axes = 'czyx'
#frames.default_coords['m'] = 0

In [ ]:
from skimage import data
import napari
#res8 = (res_stack/256).astype('uint8')
with napari.gui_qt():
    viewer = napari.Viewer()
    viewer.add_image(res[1].data
                     , colormap = 'red'
                    )
    viewer.add_image(res[0].data
                     ,colormap = 'blue'
                    )
#     viewer.add_image(frames[0]
#                      , colormap = 'red'
#                     )
#     viewer.add_image(frames[2]
#                      ,colormap = 'blue'
#                     )
#     ctrl.default_coords['c'] = 1
#     viewer.add_image(ctrl[0]
#                      , colormap = 'red'
#                     )
#     ctrl.default_coords['c'] = 0
#     viewer.add_image(ctrl[0]
#                      ,colormap = 'blue'
#                     )

In [ ]:
res[1].data.shape

In [ ]:
ctrl[0].data.shape